In [32]:
import cv2
import numpy as np
import os
import pandas as pd
import re

In [33]:
##### Modify here #####
# Folder Structure:
#   Condition
#       |
#       ----- Replicate 1-x
#                   |
#                   ------- Fiji-Macro - generated Image Folders

#condition_folder = '/Volumes/msdata/flwilfli/Lab/users/Jeremy/Nikon/GFP-Ede1-GFP+3xGBP_20h_Rapa_starvation/20hRapa/'
condition_folder = r'/Volumes/msdata/flwilfli/Lab/users/Jeremy/Nikon/GFP-Ede1-GFP+3xGBP_20h_Rapa_starvation/20hRapa/'

#################

#Switch this
replicate_folder = [name for name in os.listdir(condition_folder) if os.path.isdir(os.path.join(condition_folder,name))]
#replicate_folder = ["Replicate 1"]


roi_results = pd.DataFrame(columns=['Replicate','Cell Mask Name','Punctae Mask Name', 'Total Cells', 'Total Punctae','Cells with punctae', 'Cells w/o punctae'])

ri = 0
img_cells_mask,img_punctae_mask = {},{}
for r in replicate_folder:
    img_cells_mask[ri] = [name for name in os.listdir(os.path.join(condition_folder,r,'cells_mask')) 
    if  (os.path.join(condition_folder,r,'cells_mask',name)).endswith('mask.tif')]
    img_cells_mask[ri].sort(key = lambda s: re.findall(r'_\d+_',s))
    img_punctae_mask[ri] = [name for name in os.listdir(os.path.join(condition_folder,r,'punctae_mask')) 
    if (os.path.join(condition_folder,r,'punctae_mask',name)).endswith('nd2_mask.tif')]
    
    
    img_punctae_mask[ri].sort(key = lambda s: re.findall(r'_\d+_',s))
    
    
    #break  # <--- remove this
    ri += 1



In [34]:
def count_objects(file_path_roi, file_path_signal):
    # Read the ROI image file
    img_roi = cv2.imread(file_path_roi, cv2.IMREAD_GRAYSCALE)

    # Read the signal image file
    img_signal = cv2.imread(file_path_signal, cv2.IMREAD_GRAYSCALE)

    # Threshold the images to create binary images
    thresh_roi = cv2.threshold(img_roi, 0, 255, cv2.THRESH_BINARY)[1]
    thresh_signal = cv2.threshold(img_signal, 0, 255, cv2.THRESH_BINARY)[1]

    # Invert the binary images (to make objects white and background black)
    inverted_roi = cv2.bitwise_not(thresh_roi)
    inverted_signal = cv2.bitwise_not(thresh_signal)

    # Find contours in the inverted images
    contours_roi, hierarchy = cv2.findContours(inverted_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_signal, hierarchy = cv2.findContours(inverted_signal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Count the number of ROI contours (i.e., objects) found
    roi_count = len(contours_roi)

    # Count the number of signal contours (i.e., objects) found
    signal_count = len(contours_signal)

    # Count the number of ROI contours that have at least one signal inside
    roi_with_signal_count = 0

    for roi_contour in contours_roi:
        for signal_contour in contours_signal:
            signal_point = tuple(signal_contour[0][0].astype(float))
            if cv2.pointPolygonTest(roi_contour, signal_point, False) >= 0:
                roi_with_signal_count += 1
                break

    # Count the number of ROI contours that have no signal inside
    roi_without_signal_count = roi_count - roi_with_signal_count
    return roi_count, signal_count, roi_with_signal_count, roi_without_signal_count


In [35]:
#os.path.join(condition_folder,replicate_folder[0], roi_results.loc[0, 'Cell Mask Name'])
#roi_results.loc[1,'Cell Mask Name']                                                                 

In [36]:
for r in range(0,len(replicate_folder)):
    for c in range(0,len(img_cells_mask[r])):
        row = (r)*(len(img_cells_mask[r])) + c        
        roi_results.loc[row,'Replicate'] = replicate_folder[r]
        roi_results.loc[row, 'Cell Mask Name']       = img_cells_mask[r][c]
        roi_results.loc[row, 'Punctae Mask Name']    = img_punctae_mask[r][c]
        (roi_results.loc[row, 'Total Cells'], roi_results.loc[row,'Total Punctae'], roi_results.loc[row,'Cells with punctae'],
        roi_results.loc[row, 'Cells w/o punctae'])    = count_objects(os.path.join(condition_folder,replicate_folder[r],'cells_mask', roi_results.loc[row, 'Cell Mask Name']) ,
                                                                     os.path.join(condition_folder,replicate_folder[r],'punctae_mask',  roi_results.loc[row, 'Punctae Mask Name']) )
  #  break  # <--- remove this
print("ROI Count: ", roi_results['Total Cells'])
print("Signal Count: ", roi_results['Total Punctae'])
print("ROI with Signal Count: ",roi_results['Cells with punctae'])
print("ROI without Signal Count: ", roi_results['Cells w/o punctae'])

roi_results.to_excel(os.path.join(condition_folder,replicate_folder[r],'Final_really_hopefully_result.xlsx'))

ROI Count:  0      64
1      77
2     102
3     116
4      85
5     122
6      94
7      73
8      71
9      59
10     78
Name: Total Cells, dtype: object
Signal Count:  0     137
1      76
2      91
3     112
4     122
5     134
6     100
7      91
8      99
9     119
10    139
Name: Total Punctae, dtype: object
ROI with Signal Count:  0      3
1      6
2      5
3     10
4     13
5     13
6      8
7      4
8      7
9      5
10    10
Name: Cells with punctae, dtype: object
ROI without Signal Count:  0      61
1      71
2      97
3     106
4      72
5     109
6      86
7      69
8      64
9      54
10     68
Name: Cells w/o punctae, dtype: object
